In [1]:
import datetime

import os
import pickle
import time

import pandas as pd

from dotenv import load_dotenv

# import cProfile
# import collections
# import numpy as np
# from datetime import datetime
# from dotenv import load_dotenv

import gc


pd.options.display.float_format = "{:.2f}".format
# plt.rcParams.update({'font.size': 18})
format = '%d/%m/%Y %H:%M'

In [2]:
load_dotenv()
SPOTIPY_API = os.getenv('SPOTIPY_API')
SPOTIPY_API_KEY = os.getenv('SPOTIPY_API_KEY')
scope=''

# https://spotipy.readthedocs.io/en/latest/#
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_API, client_secret=SPOTIPY_API_KEY)

# help(sp)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager,requests_timeout=300)

# 1. Get user and playlist IDs

In [3]:
n = 950 # max = 950
p_limit = 50 # limit of playlists max = 50
years = range(2018,2022,1)

In [4]:
# Extract the user and playlist IDs
format = '%d/%m/%Y %H:%M'
t = time.time()

# check if data extraction already exists then load if it exists, otherwise performs the data request from API and save the file
data_path = os.getcwd() + '\\data\\external\\raw_1_uid_pid.pkl'
if os.path.exists(data_path):
    print('Playlist ID and User ID - Data already requested from API - Loaded: ' + data_path)
    # load file only if the next step was not executed (getting tracks)
    data_path_2 = os.getcwd() + '\\data\\external\\raw_2_playlists.pkl'
    if not(os.path.exists(data_path_2)):
        with open(data_path, 'rb') as f:
            uid_pid_raw = pickle.load(f)
else:
    print('Playlist ID and User ID - Data requested from API - Saved: ' + data_path)
    # list of all user ids (UID) and playlist ids (PID)
    uid_pid_raw = []
    
    for y in years:
        txt = 'year:' + str(y)
        print(txt)
        for i in range(0,n,1):
            if i % 100 == 0:
                t_now = datetime.datetime.now().strftime(format)
                calc_eta = datetime.datetime.now() + datetime.timedelta(seconds = n - i) * (time.time() - t) / (i + 1)
                calc_eta = calc_eta.strftime(format)
                print("%s - %d/%d elapsed time (min): %.1f - ETA: %s" % (t_now, i, n, (time.time()-t)/60 ,calc_eta))
            uid_pid = sp.search(q=txt, type='playlist', limit=p_limit, offset=i,market='US')
            uid_pid_raw.append(uid_pid)
    # Save the data
    with open(data_path, 'wb') as f:
        pickle.dump(uid_pid_raw, f)
        
print(len(uid_pid_raw)) # CHECK WHY THE COUNT OF PLAYLISTS DECREASED

Playlist ID and User ID - Data already requested from API - Loaded: C:\Users\map_f\OneDrive\Documents\GitHub\Follow_the_beat\data\external\raw_1_uid_pid.pkl
3800


# 2. Get the details of each playlist

In [5]:
# extract = True
extract = False

# Get the detail of all playlists
format = '%d/%m/%Y %H:%M'
t = time.time()

n = len(uid_pid_raw)
data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_N.pkl'
if extract == True:
    print('Details of each playlist - Data requested from API - Saved:' + data_path)
    # Get details of all playlists
    playlists_full = []
    print('Getting details of all playlists')
    for i, uid_pid in enumerate(uid_pid_raw):
        if i % 100 == 0 and i > 0:
            playlists_full = []
            t_now = datetime.datetime.now().strftime(format)
            calc_eta = datetime.datetime.now() + datetime.timedelta(seconds = n - i) * (time.time() - t) / (i + 1)
            calc_eta = calc_eta.strftime(format)
            print("%s - %d/%d elapsed time (min): %.1f - ETA: %s" % (t_now, i, n, (time.time()-t)/60 ,calc_eta))

        for user in uid_pid['playlists']['items']:
            pid = user['id']
            uid = user['owner']['id']
            playlist = sp.user_playlist(uid, pid)
            playlists_full.append(playlist)

        # Save the data
        if i % 100 == 0 and i > 0:
            print(i)
            data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_' + str(int(i/100)) + '.pkl'
            print(data_path)
            with open(data_path, 'wb') as f:
                pickle.dump(playlists_full, f)

# 2. Concatenate all the playlists, split between test and train sets, and create dictionaries for the main features data 

## 2.1 Raw playlists

In [6]:
# Concatenate all playlists
data_path = os.getcwd() + '\\data\\external\\raw_2_playlists_full.pkl'
txt = 'All playlists concatenated - ' + data_path
if os.path.exists(data_path):
    print('Loaded: ' + txt)
    with open(data_path, 'rb') as f:
        playlists_full_by_user = pickle.load(f)
else:
    print('Saved: ' + txt)
    playlists_full_by_user = []
    for i in range(0,100,1):
        data_path_aux = os.getcwd() + '\\data\\external\\raw_2_playlists_' + str(i) + '.pkl'
        if os.path.exists(data_path_aux):
            print(data_path_aux)
            with open(data_path_aux, 'rb') as f:
                playlist = pickle.load(f)
            playlists_full_by_user.extend(playlist)

    with open(data_path, 'wb') as f:
        pickle.dump(playlists_full_by_user, f)

print(len(playlists_full_by_user))

Loaded: All playlists concatenated - C:\Users\map_f\OneDrive\Documents\GitHub\Follow_the_beat\data\external\raw_2_playlists_full.pkl
2180


## 2.2 Create dictionary for all Playlists

In [7]:
def get_artists(raw_artist):
    artists = []
    for a in raw_artist:
        artists.append({'id': a['id'],
                'name': a['name']})
    return(artists);

# a = get_artists(playlists_full_by_user[6]['tracks']['items'][2]['track']['album']['artists'])
# a

In [8]:
def get_track_details(raw_track):
    if raw_track['track'] is not None:
        artists = get_artists(raw_track['track']['album']['artists'])
        track = raw_track['track']
        album = {
                'id': track['album']['id'],
                'name': track['album']['name'],
                'release_date': track['album']['release_date']
                }
        data = {
                    'id': track['id'],
                    'name': track['name'],
                    'added_at': raw_track['added_at'],
                    'popularity': track['popularity'],
                    'artists': artists,
                    'album': album
                    }
        return(data);

# a = get_track_details(playlists_full_by_user[6]['tracks']['items'][82])
# a['album']
# a = get_track_details(playlists_full_by_user[6]['tracks']['items'][83])
# playlists_full_by_user[6]['tracks']['items'][83]

In [9]:
def get_playlist_detail(playlist):
    tracks = []
    for i, item in enumerate(playlist['tracks']['items']):
        if not(item['track'] is None or item['track']['id'] is None):
            track = get_track_details(item)
            tracks.append(track)
    data = {
                'user_id': playlist['owner']['id'],
                'playlist_id': playlist['id'],
                'total_tracks': len(tracks),
                'tracks': tracks
        }

    return(data);
# t = get_playlist_detail(playlists_full_by_user[4])
# t['tracks'][0]

In [10]:
gc.collect();
# Concatenate all playlists
data_path = os.getcwd() + '\\data\\external\\raw_3_playlists_dict.pkl'
txt = 'All playlists as dictionary - ' + data_path
if os.path.exists(data_path):
    print('Loaded: ' + txt)
    with open(data_path, 'rb') as f:
        playlists_dict = pickle.load(f)
else:
    # Concatenate all playlists as dictionary
    n = len(playlists_full_by_user)

    playlists_dict = {}
    for i, playlist in enumerate(playlists_full_by_user):
        if i % 100 == 0:
            print("%s - %d/%d - LEN: %d" % (datetime.datetime.now().strftime(format), i, n, len(playlists_dict)))

        play_dict = get_playlist_detail(playlist)
        playlists_dict.update({playlist['id']: play_dict})

    print('Saved: ' + txt)
    with open(data_path, 'wb') as f:
        pickle.dump(playlists_dict, f)

print(len(playlists_dict))  # CHECK WHY THE COUNT OF PLAYLISTS DECREASED: DUPLICATED PLAYLISTS

Loaded: All playlists as dictionary - C:\Users\map_f\OneDrive\Documents\GitHub\Follow_the_beat\data\external\raw_3_playlists_dict.pkl
1851


In [11]:
cnt = []
n = len(playlists_full_by_user)
for i, playlist in enumerate(playlists_full_by_user):
#     if i % 100 == 0:
#         print("%s - %d/%d - LEN: %d" % (datetime.datetime.now().strftime(format), i, n, len(playlists_full_by_user)))

    play_dict = get_playlist_detail(playlist)
    cnt.append(playlist['id'])
    
df = pd.DataFrame (cnt, columns = ['id'])
count = pd.DataFrame(df.groupby('id').size(),columns=['cnt'])
a = count.groupby('cnt').size()
print('Count of xyz')
print(a)
a.sum()

Count of xyz
cnt
1    1540
2     296
3      12
4       3
dtype: int64


1851